# モデルを監視する

モデルをサービスとして運用環境にデプロイしたら、モデルを監視して使用状況を追跡し、処理する要求を調査します。このラボでは、Azure Application Insights を使用して、モデル サービス エンドポイントのアクティビティを監視します。


## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## 展開用モデルを準備する

次に、デプロイするモデルが必要です。次のコードを実行します。

1. データセットを作成して登録します。
2. データセットを使用してモデルをトレーニングします。
3. モデルを登録します。

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from azureml.core import Dataset

# データ ファイルを既定のデータストアにアップロードする
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-data/',
                       overwrite=True,
                       show_progress=True)

# データストア上のパスから表形式のデータセットを作成する
print('Creating dataset...')
data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 表形式のデータセットを登録する
print('Registering dataset...')
data_set = data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

# ワークスペースで Azure 実験を作成する
experiment = Experiment(workspace = ws, name = "diabetes-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 糖尿病データセットを読み込む
print("Loading Data...")
diabetes = data_set.to_pandas_dataframe()

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# トレーニング済みモデルを保存する
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# 実行を完了する
run.complete()

# モデルを登録する
print('Registering model...')
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 登録済みモデルを取得する
model = ws.models['diabetes_model']

print('Model trained and registered.')

## モデルを Web サービスとしてデプロイする

これで、登録済みモデルを Web サービスとしてデプロイする準備が整いました。

最初に、展開構成ファイル用フォルダーを作成します

In [ ]:
import os

folder_name = 'diabetes_service'
os.makedirs(folder_name, exist_ok=True)
print(folder_name)

次に、サービスが新しいデータのスコアリングに使用するエントリ スクリプトが必要です。

In [ ]:
%%writefile $folder_name/score_diabetes.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

# サービスの読み込み時に呼び出される
def init():
    global model
    # デプロイ済みのモデル ファイルへのパスを取得して読み込む
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# 要求の受信時に呼び出される
def run(raw_data):
    # 入力データを numpy 配列として取得する
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    # モデルから予測を取得する
    predictions = model.predict(np_data)
    # データと予測を印刷する (ログに記録されます!)。
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    # 各予測に対応するクラス名を取得する (0 または 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # 予測を JSON 形式で返す
    return json.dumps(predicted_classes)

また、サービス環境用の Conda 構成ファイルも必要です。

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# モデルの依存関係を追加する (AzureML の既定値は既に含まれています)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# 環境構成を .yml ファイルとして保存する
env_file = folder_name + "/diabetes_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# .yml ファイルを印刷する
with open(env_file,"r") as f:
    print(f.read())

これで、サービスをデプロイできます (この場合は Azure Container Instances (ACI) として)。

> **注**: これには数分かかる場合があります。状態が**正常**と表示されるまで待ちます。

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig

# スコアリング環境を構成する
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_diabetes.py",
                                   conda_file="diabetes_env.yml")

service_name = "diabetes-service-app-insights"
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aci_service = Model.deploy(workspace=ws,
                           name= service_name,
                           models= [model],
                           inference_config= inference_config,
                           deployment_config=deployment_config)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

## Application Insights を有効にする

次に、サービスの Application Insights 有効にする必要があります。

In [ ]:
# AppInsights を有効にする
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

## Web サービスを使用する

サービスをデプロイしたら、クライアント アプリケーションからサービスを使用できます。

まず、これらのアプリケーションが要求を送信する URL を決定します。

In [ ]:
endpoint = aci_service.scoring_uri
print(endpoint)

エンドポイント URI がわかったので、アプリケーションは HTTP 要求を行い、患者データを JSON (またはバイナリ) 形式で送信し、予測されたクラスを受け取ることができます。

> **ヒント**: サービス エンドポイントの準備が整っていないためエラーが発生した場合。数秒待ってからもう一度やり直してください!

In [ ]:
import requests
import json

# 推論用の新しいデータを作成する
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# JSON ドキュメント内のシリアル化可能なリストに配列を変換する
input_json = json.dumps({"data": x_new})

# コンテンツ タイプを設定する
headers = { 'Content-Type':'application/json' }

# 予測を取得する
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )

これで、サービス エンドポイントにについて記録されたデータを表示できます。
1. [Azure portal](https://portal.azure.com) で、Machine Learning ワークスペースを開きます。
2. **概要**ページで、関連付けられている **Application Insights** リソースのリンクをクリックします。
3. Application Insights ブレードで、**ログ (分析)** をクリックします。 

    > **注意**: 初めてログ分析を開く場合は、**はじめに**をクリックして、クエリ エディターを開く必要があります。クエリの書き込み方法を説明するヒントが表示されたら、それを閉じます。

4. 次のクエリをクエリ ウィンドウに貼り付けて、**実行**をクリックします
    ```
    traces
    |where  message == "STDOUT"
      and customDimensions.["Service Name"] == "diabetes-service-app-insights"
    |project timestamp, customDimensions.Content
    ```
5. 結果を表示します。ACI Web サービスは、テレメトリを Application Insights に送信するのに 2 ～3 分かかる場合があるため、最初は何も存在しない場合があります。数分待ってから、ログに記録されたデータと予測が表示されるまでクエリを再実行します。

## サービスを削除する

サービスが不要になった場合は、不要な料金が発生しないように削除する必要があります。

In [ ]:
aci_service.delete()

Application Insights を使用してデプロイされたサービスを監視する方法の詳細については、[Azure Machine Learning のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-enable-app-insights)を参照してください。